## Practice lab: Deep Learning for Content-Based Filtering

## Outline
- [ 1 - Packages](#1)
- [ 2 - Movie ratings dataset](#2)
- [ 3 - Neural Network for content-based filtering](#3)
- [ 4 - Model Analysis](#4)
- [ 5 - Finding Similar Items](#5)

<a name="1"></a>
## 1 - Packages

In [22]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)

<a name="2"></a>
## 2 - Movie ratings dataset
The data set is derived from the [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/) dataset. 

In [23]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
scaledata = True  # applies the standard scalar to data if true
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 58187


In [24]:
#Some of the user and item/movie features are not used in training. 
# Below, the features in brackets "[]" such as the "user id", "rating count" and "rating ave" are not included when the model is trained and used. 
# Note, the user vector is the same for all the movies rated.
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9


In [25]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8798,2004,3.8,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [26]:
# scale training data
if scaledata:
    item_train_save = item_train
    user_train_save = user_train

    scalerItem = StandardScaler()
    scalerItem.fit(item_train)
    item_train = scalerItem.transform(item_train)

    scalerUser = StandardScaler()
    scalerUser.fit(user_train)
    user_train = scalerUser.transform(user_train)

    print(np.allclose(item_train_save, scalerItem.inverse_transform(item_train)))
    print(np.allclose(user_train_save, scalerUser.inverse_transform(user_train)))

True
True


In [27]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test  data shape: {item_test.shape}")

movie/item training data shape: (46549, 17)
movie/item test  data shape: (11638, 17)


In [28]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
1,0,0.6,0.7,0.6,0.6,0.7,0.7,0.5,0.7,0.2,0.3,0.3,0.5,0.5,0.8,0.5
0,0,1.6,1.5,1.7,0.9,1.0,1.4,0.8,-1.2,1.2,1.2,1.6,0.9,1.4,1.2,1.0
0,0,0.8,0.6,0.7,0.5,0.6,0.6,0.3,-1.2,0.7,0.8,0.9,0.6,0.2,0.6,0.6
1,0,-0.1,0.2,-0.1,0.3,0.7,0.3,0.2,1.0,-0.5,-0.7,-2.1,0.5,0.7,0.3,0.0
-1,0,-1.3,-0.8,-0.8,0.1,-0.1,-1.1,-0.9,-1.2,-1.5,-0.6,-0.5,-0.6,-0.9,-0.4,-0.9


In [29]:
#Scaling the target ratings using a Min Max Scaler to scale the target to be between -1 and 1
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

(46549, 1) (11638, 1)


<a name="3"></a>
## 3 - Neural Network for content-based filtering

<figure>
    <center> <img src="./images/RecSysNN.png"   style="width:500px;height:280px;" ></center>
</figure>

In [30]:

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = Model([input_user, input_item], output)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 14)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 sequential_2 (Sequential)      (None, 32)           40864       ['input_3[0][0]']                
                                                                                                  
 sequential_3 (Sequential)      (None, 32)           41376       ['input_4[0][0]']                
                                                                                            

In [31]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [32]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], ynorm_train, epochs=30)

Epoch 1/30
1455/1455 [==============================] - 6s 3ms/step - loss: 0.1254
Epoch 2/30
1455/1455 [==============================] - 4s 3ms/step - loss: 0.1183
Epoch 3/30
1455/1455 [==============================] - 4s 3ms/step - loss: 0.1167
Epoch 4/30
1455/1455 [==============================] - 4s 2ms/step - loss: 0.1150
Epoch 5/30
1455/1455 [==============================] - 4s 3ms/step - loss: 0.1140
Epoch 6/30
1455/1455 [==============================] - 4s 3ms/step - loss: 0.1126
Epoch 7/30
1455/1455 [==============================] - 4s 3ms/step - loss: 0.1118
Epoch 8/30
1455/1455 [==============================] - 4s 3ms/step - loss: 0.1111
Epoch 9/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1101
Epoch 10/30
1455/1455 [==============================] - 4s 3ms/step - loss: 0.1092
Epoch 11/30
1455/1455 [==============================] - 4s 3ms/step - loss: 0.1076
Epoch 12/30
1455/1455 [==============================] - 4s 2ms/step - loss: 0.1071
E

<a name="4"></a>
## 4 - Model Analysis

In [33]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], ynorm_test)

364/364 [==============================] - 1s 2ms/step - loss: 0.1050


0.10502442717552185

#### Predictions for a new user

In [34]:

new_user_id = 5000
new_rating_ave = 1.0
new_action = 1.0
new_adventure = 1
new_animation = 1
new_childrens = 1
new_comedy = 5
new_crime = 1
new_documentary = 1
new_drama = 1
new_fantasy = 1
new_horror = 1
new_mystery = 1
new_romance = 5
new_scifi = 5
new_thriller = 1
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [35]:
# generate and replicate the user vector to match the number movies in the data set
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale the vectors and make predictions for all movies. Return results sorted by rating
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs,  item_vecs, model, u_s, i_s, 
                                                                       scaler, scalerUser, scalerItem, scaledata=scaledata)

print_pred_movies(sorted_ypu, sorted_user, sorted_items, movie_dict, maxcount = 10)

59/59 [==============================] - 0s 2ms/step


y_p,movie id,rating ave,title,genres
4.84988,76293,3.31818,Date Night (2010),Action|Comedy|Romance
4.82937,5377,3.71591,About a Boy (2002),Comedy|Drama|Romance
4.82872,62155,3.35,Nick and Norah's Infinite Playlist (2008),Comedy|Drama|Romance
4.82528,56949,3.3,27 Dresses (2008),Comedy|Romance
4.82398,5791,3.78125,Frida (2002),Drama|Romance
4.82297,5992,3.7,"Hours, The (2002)",Drama|Romance
4.81791,84954,3.16667,"Adjustment Bureau, The (2011)",Romance|Sci-Fi|Thriller
4.81495,58047,3.42857,"Definitely, Maybe (2008)",Comedy|Drama|Romance
4.808,7151,3.375,Girl with a Pearl Earring (2003),Drama|Romance
4.8053,27808,3.36364,Spanglish (2004),Comedy|Drama|Romance


#### Predictions for an existing user

In [36]:
uid =  36 
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, scalerUser.inverse_transform(user_train), item_vecs, user_to_genre)

# scale the vectors and make predictions for all movies. Return results sorted by rating.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler, 
                                                                      scalerUser, scalerItem, scaledata=scaledata)
sorted_y = y_vecs[sorted_index]

#print sorted predictions
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, item_features, ivs, uvs, movie_dict, maxcount = 10)

59/59 [==============================] - 0s 2ms/step


y_p,y,user,user genre ave,movie rating ave,title,genres
3.3,3.0,36,3.00,2.86,"Time Machine, The (2002)",Adventure
3.2,3.0,36,3.00,2.86,"Time Machine, The (2002)",Action
3.2,3.0,36,3.00,2.86,"Time Machine, The (2002)",Sci-Fi
2.1,1.0,36,1.00,4.00,"Beautiful Mind, A (2001)",Romance
2.0,2.0,36,1.75,3.52,Gangs of New York (2002),Crime
2.0,1.5,36,1.75,3.52,Road to Perdition (2002),Crime
2.0,1.0,36,1.50,4.00,"Beautiful Mind, A (2001)",Drama
1.9,1.5,36,1.50,3.52,Road to Perdition (2002),Drama
1.9,2.0,36,1.50,3.52,Gangs of New York (2002),Drama


<a name="5"></a>
## 5 - Finding Similar Items

In [37]:
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """   
    d = sum(np.square(a-b)) 
    return (d)


A matrix of distances between movies can be computed once when the model is trained and then reused for new recommendations without retraining. The first step, once a model is trained, is to obtain the movie feature vector, $v_m$, for each of the movies. To do this, we will use the trained `item_NN` and build a small model to allow us to run the movie vectors through it to generate $v_m$.

In [38]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 16)]              0         
                                                                 
 sequential_3 (Sequential)   (None, 32)                41376     
                                                                 
 tf.math.l2_normalize_4 (TFO  (None, 32)               0         
 pLambda)                                                        
                                                                 
Total params: 41,376
Trainable params: 41,376
Non-trainable params: 0
_________________________________________________________________


In [39]:
#creating a set of movie feature vectors
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

59/59 [==============================] - 0s 4ms/step
size of all predicted movie feature vectors: (1883, 32)


#### Similar Movies

In [40]:
count = 50
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    genre1,_  = get_item_genre(item_vecs[i,:], ivs, item_features)
    genre2,_  = get_item_genre(item_vecs[min_idx,:], ivs, item_features)

    disp.append( [movie_dict[movie1_id]['title'], genre1,
                  movie_dict[movie2_id]['title'], genre2]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=[".1f", ".1f", ".0f", ".2f", ".2f"])
table

movie1,genres,movie2,genres
Save the Last Dance (2001),Drama,John Q (2002),Drama
Save the Last Dance (2001),Romance,"Sweetest Thing, The (2002)",Romance
"Wedding Planner, The (2001)",Comedy,Mr. Deeds (2002),Comedy
"Wedding Planner, The (2001)",Romance,Mr. Deeds (2002),Romance
Hannibal (2001),Horror,Final Destination 2 (2003),Horror
Hannibal (2001),Thriller,"Sum of All Fears, The (2002)",Thriller
Saving Silverman (Evil Woman) (2001),Comedy,Cats & Dogs (2001),Comedy
Saving Silverman (Evil Woman) (2001),Romance,Save the Last Dance (2001),Romance
Down to Earth (2001),Comedy,Joe Dirt (2001),Comedy
Down to Earth (2001),Fantasy,"Haunted Mansion, The (2003)",Fantasy
